Code to process the .dat file optained from the scanner 
This shoudl be run 1 direction at a time 

The intial loading of the file takes around 10 minutes per direction - this only needs to be ran once for coil combination - no significant run times after this
This file performs the coil combination based on the reference data and then separates the data into the relevant .mat files
Remember to add the InputGradients.mat file into the folder for the GIRF calcualtion 

In [1]:
import numpy as np
from mapvbvd import mapVBVD as mapVBVD
import matplotlib.pyplot as plt
import twixtools 
from fsl_mrs.utils.preproc import combine
from scipy.io import savemat
from fsl_mrs.utils.preproc.combine import svd_reduce, weightedCombination

In [2]:
def process_mri_data(pattern, filename, start_idx=0, num_scans_to_load=3500):
    # Load the Twix file with parsed data
    multi_twix = twixtools.read_twix(filename, parse_data=True)
    
    # Extract the subset of mdb objects
    end_idx = start_idx + num_scans_to_load
    mdb_data_subset = multi_twix[-1]['mdb'][start_idx:end_idx]
    
    # Initialize a list to hold the gradient data
    all_data = []
    
    # Extract data for the subset
    for mdb in mdb_data_subset:
        if mdb.is_image_scan():  # Check if it's an image scan
            all_data.append(mdb.data)
    
    # Convert the list to a NumPy array
    all_data = np.asarray(all_data)
    
    # Reshape and transpose data
    #This is because the 80000 data points are collected as 10 groups of 8000
    ungrouped = all_data.reshape(350, 10, 32, 8000)
    ungroupedT = np.transpose(ungrouped, (0, 2, 1, 3))
    gradient_data = ungroupedT.reshape(350, 32, 80000)
    data = np.transpose(gradient_data, (0, 2, 1))
    
    # Split data into reference and triangle groups
    data_ref = []
    data_tri = []
    
    RRef = 1 #Number of repeats which shouldve been left as 1
    RTri = 1
    chunk_size = 50 * RRef + 300 * RTri
    for i in range(0, np.shape(data)[0], chunk_size):
        data_ref.append(data[i:i + 50 * RRef, :, :])
        data_tri.append(data[i + 50 * RRef:i + chunk_size, :, :])
    
    data_ref = np.vstack(data_ref)
    data_tri = np.vstack(data_tri)
    
    # Process reference data
    averaged_data_ref = data_ref.reshape(50, RRef, 80000, 32).mean(axis=1)
    
    #Apply coil combination to the reference data and calcuate the weights for the triangle data 
    data_ref_corrected = []
    weightslist = []
    for i in range(np.shape(averaged_data_ref)[0]):
        combined, weights, _ = svd_reduce(averaged_data_ref[i], return_alpha=True)
        data_ref_corrected.append(combined)
        weightslist.append(weights)
    
    data_ref_corrected = np.array(data_ref_corrected)
    weightslist = np.array(weightslist)
    
    # Apply weighting to triangle data
    data_tri_corrected = []
    for idx, element in enumerate(pattern):
        fid = weightedCombination(data_tri[idx], weightslist[element])
        data_tri_corrected.append(fid)
    
    data_tri_corrected = np.array(data_tri_corrected)
    
    return data_ref_corrected, data_tri_corrected

In [3]:
#Used to split the data into batches, perform the coil combination, then rejoin the batches together 
#Batch processing only performed to help with memory constraints running on a CPU

def process_full_mri_data(pattern, filename):
    total_scans = 21000
    num_scans_to_load = 3500 #This is a reference and 3 different triangluar amplitudes 
    num_iterations = total_scans // num_scans_to_load
    
    data_ref_corrected_list = []
    data_tri_corrected_list = []
    
    for i in range(num_iterations):
        data_ref_corrected, data_tri_corrected = process_mri_data(pattern, filename, start_idx=i * num_scans_to_load)
        data_ref_corrected_list.append(data_ref_corrected)
        data_tri_corrected_list.append(data_tri_corrected)
    
    final_data_ref_corrected = np.concatenate(data_ref_corrected_list, axis=0)
    final_data_tri_corrected = np.concatenate(data_tri_corrected_list, axis=0)
    
    return final_data_ref_corrected, final_data_tri_corrected


In [4]:
#Load in .dat file 
direction = 'z'
filename = f'/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/meas_MID00184_FID46448_full_{direction}.dat'

#The pattern is used so that the coil combination for the triangular data used the coil weights from the equivalent reference data

def generate_pattern(limit):
    pattern = []
    for i in range(0, limit + 1, 2):
        pattern.extend([i, i+1, i, i+1])
    return pattern

pattern = generate_pattern(48) # Goes from 0-49, this is the 25 phase encode directions * 2 slices (positive and negative offsets)
pattern = pattern*3            # Multiply by 3 as there are 3 triangle amplitudes between each reference and contained in each batch 
print(pattern)


[0, 1, 0, 1, 2, 3, 2, 3, 4, 5, 4, 5, 6, 7, 6, 7, 8, 9, 8, 9, 10, 11, 10, 11, 12, 13, 12, 13, 14, 15, 14, 15, 16, 17, 16, 17, 18, 19, 18, 19, 20, 21, 20, 21, 22, 23, 22, 23, 24, 25, 24, 25, 26, 27, 26, 27, 28, 29, 28, 29, 30, 31, 30, 31, 32, 33, 32, 33, 34, 35, 34, 35, 36, 37, 36, 37, 38, 39, 38, 39, 40, 41, 40, 41, 42, 43, 42, 43, 44, 45, 44, 45, 46, 47, 46, 47, 48, 49, 48, 49, 0, 1, 0, 1, 2, 3, 2, 3, 4, 5, 4, 5, 6, 7, 6, 7, 8, 9, 8, 9, 10, 11, 10, 11, 12, 13, 12, 13, 14, 15, 14, 15, 16, 17, 16, 17, 18, 19, 18, 19, 20, 21, 20, 21, 22, 23, 22, 23, 24, 25, 24, 25, 26, 27, 26, 27, 28, 29, 28, 29, 30, 31, 30, 31, 32, 33, 32, 33, 34, 35, 34, 35, 36, 37, 36, 37, 38, 39, 38, 39, 40, 41, 40, 41, 42, 43, 42, 43, 44, 45, 44, 45, 46, 47, 46, 47, 48, 49, 48, 49, 0, 1, 0, 1, 2, 3, 2, 3, 4, 5, 4, 5, 6, 7, 6, 7, 8, 9, 8, 9, 10, 11, 10, 11, 12, 13, 12, 13, 14, 15, 14, 15, 16, 17, 16, 17, 18, 19, 18, 19, 20, 21, 20, 21, 22, 23, 22, 23, 24, 25, 24, 25, 26, 27, 26, 27, 28, 29, 28, 29, 30, 31, 30, 31, 32,

In [5]:
data_ref_corrected, data_tri_corrected = process_full_mri_data(pattern=pattern, filename=filename) #Takes ~10 mins 

Software version: VD/VE (!?)

Scan  0


  9%|▉         | 3.66G/40.1G [00:00<00:08, 4.76GB/s]/opt/anaconda3/envs/fsl_mrs/lib/python3.11/site-packages/tqdm/std.py:1229: RuntimeWarning: overflow encountered in scalar add
  self.n += n
/opt/anaconda3/envs/fsl_mrs/lib/python3.11/site-packages/tqdm/std.py:1232: RuntimeWarning: overflow encountered in scalar subtract
  if self.n - self.last_print_n >= self.miniters:
/opt/anaconda3/envs/fsl_mrs/lib/python3.11/site-packages/tqdm/std.py:1237: RuntimeWarning: overflow encountered in scalar subtract
  dn = self.n - self.last_print_n  # >= n
  0%|          | 80.0M/40.1G [00:07<1:07:41, 10.6MB/s]


Software version: VD/VE (!?)

Scan  0


  0%|          | 80.0M/40.1G [00:07<1:07:47, 10.6MB/s]


Software version: VD/VE (!?)

Scan  0


...

  0%|          | 80.0M/40.1G [00:07<1:01:38, 11.6MB/s]


Software version: VD/VE (!?)

Scan  0


  0%|          | 80.0M/40.1G [00:07<1:01:05, 11.7MB/s]


Software version: VD/VE (!?)

Scan  0


  0%|          | 80.0M/40.1G [00:07<1:02:20, 11.5MB/s]


In [6]:
print(data_ref_corrected.shape)
print(data_tri_corrected.shape)

(300, 80000)
(1800, 80000)


In [12]:
data_ref = data_ref_corrected.reshape(6, 50, 80000)

# Split into positive and negative y-slices (shape: 80000, 25, 6)
dataposy = data_ref[:, 0::2, :].transpose(2, 1, 0)  # Even indices correspond to a positive slice offset 
datanegy = data_ref[:, 1::2, :].transpose(2, 1, 0)  # Odd indices correspond to a negative slice offset
direction ='z'

#Apply the 2D FT for the Phase encoding 
def ref_2dFT(input):
    input_grid = input.reshape(80000, 5, 5, 6)
    ft_kspace = ft_kspace = np.fft.fft2(input_grid, axes=(1, 2))
    ft_kspace_shifted = np.fft.fftshift(ft_kspace, axes=(1, 2))
    modified_kspace_all = ft_kspace_shifted.reshape(80000, 25, 6)

    return modified_kspace_all

dataposy_FT = ref_2dFT(dataposy)
datanegy_FT = ref_2dFT(datanegy)

# Experiment parameters
RRef = 1
RTri = 1
acqNumRef = 6
acqNumTri = 18 
dwellTime = 5 #In microseconds

gradAmp = np.array([
    9, 10.8, 12.6, 14.4, 16.2, 18, 19.8, 21.6, 23.4, 25.2, 
    27, 28.8, 30.6, 32.4, 34.2, 36, 37.8, 39.6
]).reshape(1, 18)
nch = 1 #As we have alreayd performed coil combination 
roPts = 80000
roTime = np.arange(0, dwellTime * roPts, dwellTime)

# Save positive y-slice
MatDataRef_pos = {
    'acqNum': acqNumRef,
    'avgNum': RRef,
    'dwellTime': dwellTime,
    'gradAmp': gradAmp,
    'kspace_all': dataposy_FT,  
    'nch': nch,
    'roPts': roPts,
    'roTime': roTime
}
filename_pos = f"/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Ref+{direction}slice.mat"
#savemat(filename_pos, MatDataRef_pos)

# Save negative y-slice
MatDataRef_neg = {
    'acqNum': acqNumRef,
    'avgNum': RRef,
    'dwellTime': dwellTime,
    'gradAmp': gradAmp,
    'kspace_all': datanegy_FT,  
    'nch': nch,
    'roPts': roPts,
    'roTime': roTime
}
filename_neg = f"/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Ref-{direction}slice.mat"
#savemat(filename_neg, MatDataRef_neg)

In [11]:
reshaped_data = data_tri_corrected.reshape(18, 100, 80000)

# Initialize a list to hold the 4 groups
groups = []

# File paths for saving the .mat files
file_paths = [
    f'/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Positive+{direction}slice.mat',
    f'/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Positive-{direction}slice.mat',
    f'/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Negative+{direction}slice.mat',
    f'/Users/jamesbacon/Library/CloudStorage/OneDrive-Nexus365/GIRF_Correction/250313_pe_girf/GIRF_PE/Negative-{direction}slice.mat'
]

def tri_2dFT(input):
    input_grid = input.reshape(80000, 5, 5, 18)
    ft_kspace = ft_kspace = np.fft.fft2(input_grid, axes=(1, 2))
    ft_kspace_shifted = np.fft.fftshift(ft_kspace, axes=(1, 2))
    modified_kspace_all = ft_kspace_shifted.reshape(80000, 25, 18)

    return modified_kspace_all

# Process each group and save as individual .mat files
for i in range(4):
    # For each group, select the appropriate readouts based on the alternating pattern (0,1,2,3,...)
    selected_data = reshaped_data[:, i::4, :]
    
    # Now, we need to reshape this selected data into the shape (80000, 25, 18)
    group_data = selected_data.transpose(2, 1, 0)  # This gives us the shape (80000, 25, 18)

    group_data_FT = tri_2dFT(group_data)
    
    # Create the dictionary with metadata
    MatDataTri = {
        'acqNum': acqNumTri,
        'avgNum': RTri,
        'dwellTime': dwellTime,
        'gradAmp': gradAmp,
        'kspace_all': group_data_FT,  # kspace array is now the group data
        'nch': nch,
        'roPts': roPts,
        'roTime': roTime
    }
    
    # Save the dictionary along with the group data in the corresponding .mat file
    #savemat(file_paths[i],  MatDataTri)
